In [77]:
import torch
import torch.nn as nn
import torchvision

In [78]:
mnist_train = torchvision.datasets.FashionMNIST('./data', train=True, download=True)
x_train = mnist_train.data.reshape(-1, 1, 28, 28).float()  # torch.functional.nn.conv2d argument must include channels (1)
y_train = torch.zeros((mnist_train.targets.shape[0], 10))  # Create output tensor
y_train[torch.arange(mnist_train.targets.shape[0]), mnist_train.targets] = 1  # Populate output

mnist_test = torchvision.datasets.FashionMNIST('./data', train=False, download=True)
x_test = mnist_test.data.reshape(-1, 1, 28, 28).float()  # torch.functional.nn.conv2d argument must include channels (1)
y_test = torch.zeros((mnist_test.targets.shape[0], 10))  # Create output tensor
y_test[torch.arange(mnist_test.targets.shape[0]), mnist_test.targets] = 1  # Populate output


In [79]:
# Normalization of inputs
mean = x_train.mean()
std = x_train.std()
# Standard score = x - mean / std
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

# Divide training data into batches to speed up optimization
batches = 600
x_train_batches = torch.split(x_train, batches)
y_train_batches = torch.split(y_train, batches)

In [80]:
class SequentialCNNModel(nn.Module):
	def __init__(self):
		super(SequentialCNNModel, self).__init__()

		self.logits = nn.Sequential(
		nn.Conv2d(1, 32, 5, padding=2), 
		nn.ReLU(),
		nn.MaxPool2d(2), 
		nn.Conv2d(32, 64, 5, padding=2), 
		nn.MaxPool2d(2),
		nn.Conv2d(64, 16, 4, padding=2), 
		nn.Dropout(0.1),
		nn.ReLU(),
		nn.MaxPool2d(2),
		nn.Conv2d(16, 32, 4, padding=2),
		nn.MaxPool2d(2),
		# nn.Dropout(0.1),
		nn.Flatten(),
		nn.Linear(32 * 2 * 2, 1024), 
		nn.Linear(1024, 10))
	
	def f(self, x):
		return torch.softmax(self.logits(x), dim=1)
	
	def loss(self, x, y):
		return nn.functional.cross_entropy(self.logits(x), y.argmax(1))
	
	# Accuracy
	def accuracy(self, x, y):
		return torch.mean(torch.eq(self.f(x).argmax(1), y.argmax(1)).float())

In [81]:
model = SequentialCNNModel()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [82]:
for epoch in range(10):
    for batch in range(len(x_train_batches)):
        model.loss(x_train_batches[batch], y_train_batches[batch]).backward()  # Compute loss gradients
        optimizer.step()  # Perform optimization by adjusting W and b,
        optimizer.zero_grad()  # Clear gradients for next step

    print("accuracy = %s" % model.accuracy(x_test, y_test).item())

# 0.8810

accuracy = 0.8029999732971191
accuracy = 0.8442999720573425
accuracy = 0.8628000020980835
accuracy = 0.8834999799728394
accuracy = 0.8851000070571899
accuracy = 0.891700029373169
accuracy = 0.8939999938011169
accuracy = 0.899399995803833
accuracy = 0.8991000056266785
accuracy = 0.90420001745224
